In [1]:
import warnings
warnings.filterwarnings("ignore")

from copy import deepcopy
import pandas as pd
import numpy as np
import plotly.express as px
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix

from sklearn.ensemble import (
    VotingClassifier,
    RandomForestClassifier,
    AdaBoostClassifier,
    GradientBoostingClassifier
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier

import plotly.graph_objects as go


# Make datasets

In [2]:
class Dataset:
    def __init__(self, data, target):
        self.data = data
        self.target = target

In [3]:
# Assume that we alread have the data in a pickle file in backup folder
file = 'backup/category_dfs_VJ_region_frequency.pkl'
with open(file, 'rb') as file:
    data_dfs: dict = pickle.load(file)

# top 50 regions for each category
all_regions = []
for _, df in data_dfs.items():
    df = df.sort_values(by='output', ascending=False).head(50)
    regions = df['VJ_region'].values
    all_regions.extend(regions)

all_regions = list(set(all_regions))

In [4]:
# Assume that we alread have VJ dependent data in TCR_DATASETS and file TCR_load.py

from TCR_DATASETS.TCR_load import load_tcr_data
"""
if we dont have TCR_load.py, we can use the following code to load the data:

VJ_DEPENDENT = 'TCR_sequencing_hedimed_DATASET_VJ_dependent.csv'
Base_columns = {
    "patient_id": str,
    "clonotype_sequence": str,
    "specific_seq_count": int,
    "ratio": float,
    "category": str
}
VJ_DEPENDENT_COLUMNS = Base_columns.copy()
VJ_DEPENDENT_COLUMNS["V_region"] = str
VJ_DEPENDENT_COLUMNS["J_region"] = str
data_sources = { 'VJ_DEPENDENT': (VJ_DEPENDENT, VJ_DEPENDENT_COLUMNS)}

file_path, columns = data_sources[VJ_DEPENDENT]
data_df = pd.read_csv(file_path, dtype=columns, usecols=columns.keys())
"""

data_df = load_tcr_data("VJ_DEPENDENT")
data_df["VJ_region"] = data_df["V_region"] + "_" + data_df["J_region"]
data_df = data_df.drop(columns=["V_region", "J_region", "clonotype_sequence"])


# Count the number of sequences for each patient
patients_sequence_count = data_df.groupby(["patient_id"]).agg({"specific_seq_count": "sum"}).reset_index()


In [5]:
# count the frequency of each VJ region in the top 50 regions for each category
data_df = data_df.groupby(["patient_id", "category", "VJ_region"]).agg({"ratio": "sum", "specific_seq_count": "sum"}).reset_index()

# Keep only the top 50 regions
data_df = data_df[data_df["VJ_region"].isin(all_regions)]

# Normalize the data
for category in data_df["category"].unique():
    category_df = data_df[data_df["category"] == category]
    category_df["ratio"] = category_df["ratio"]
    category_df["specific_seq_count"] = category_df["specific_seq_count"] 
    data_df.loc[data_df["category"] == category, "ratio"] = category_df["ratio"]
    data_df.loc[data_df["category"] == category, "specific_seq_count"] = category_df["specific_seq_count"]


In [6]:
# make dataframe. Each row is a patient, and each column is a VJ region
dataset_freq = pd.DataFrame()
dataset_freq["patient_id"] = data_df["patient_id"].unique()

# add the category column to the dataset based on the patient_id
dataset_freq[all_regions] = 0
dataset_freq["category"] = dataset_freq["patient_id"].apply(lambda x: data_df[data_df["patient_id"] == x]["category"].values[0])

# fill the dataset with the frequency of each VJ region
for patient_id in dataset_freq["patient_id"]:
    patient_df = data_df[data_df["patient_id"] == patient_id]
    for _, row in patient_df.iterrows():
        dataset_freq.loc[dataset_freq["patient_id"] == patient_id, row["VJ_region"]] = row["ratio"]

In [7]:
# dataset sum

dataset_sum = pd.DataFrame()
dataset_sum["patient_id"] = data_df["patient_id"].unique()
dataset_sum[all_regions] = 0
dataset_sum["category"] = dataset_sum["patient_id"].apply(lambda x: data_df[data_df["patient_id"] == x]["category"].values[0])

for patient_id in dataset_sum["patient_id"]:
    patient_df = data_df[data_df["patient_id"] == patient_id]
    for _, row in patient_df.iterrows():
        dataset_sum.loc[dataset_sum["patient_id"] == patient_id, row["VJ_region"]] = row["specific_seq_count"]

## Dataset preprocessing

In [8]:
# shuffle the dataset
dataset_freq = dataset_freq.sample(frac=1, random_state=42).reset_index(drop=True)
dataset_sum = dataset_sum.sample(frac=1, random_state=42).reset_index(drop=True)

### Normalization
dataset_sum will be only binned

In [9]:
# sum the frequency of each VJ region for each patient and print
dataset_freq["sum"] = dataset_freq[all_regions].sum(axis=1)


# transform the frequency to percentage
dataset_freq[all_regions] = dataset_freq[all_regions].div(dataset_freq["sum"], axis=0)
# drop the sum column
dataset_freq = dataset_freq.drop(columns=["sum"])

### Binning

In [10]:
datasets = [dataset_freq, dataset_sum]
num_bins = 500

for dataset in datasets:
    max_value = dataset[all_regions].max().max()
    bins = np.linspace(0, max_value, num_bins) # create bins

    # digitize the data = put the data in bins
    dataset[all_regions] = np.digitize(dataset[all_regions], bins) 


### Make class Dataset

In [11]:
d_f= Dataset(
    data = dataset_freq.drop(columns=["patient_id", "category"]),
    target = dataset_freq["category"]
)

d_s = Dataset(
    data = dataset_sum.drop(columns=["patient_id", "category"]),
    target = dataset_sum["category"]
)

# make dataset without category: Enterovirus infection
dataset_freq_no_enterovirus = deepcopy(dataset_freq[dataset_freq["category"] != "Enterovirus infection"])
d_f_no_enterovirus = Dataset(
    data = dataset_freq_no_enterovirus.drop(columns=["patient_id", "category"]),
    target = dataset_freq_no_enterovirus["category"]
)


# make dataset without category: Enterovirus infection and Control

dataset_freq_no_enterovirus_control = deepcopy(dataset_freq_no_enterovirus[dataset_freq_no_enterovirus["category"] != "Control"])
d_f_no_enterovirus_control = Dataset(
    data = dataset_freq_no_enterovirus_control.drop(columns=["patient_id", "category"]),
    target = dataset_freq_no_enterovirus_control["category"]
)


# make dataset without category: Enterovirus infection. Masked category SingleAAB, MultipleAABs, and Celiac_case as Non_control

dataset_freq_no_enterovirus_control_masked = deepcopy(dataset_freq_no_enterovirus)
dataset_freq_no_enterovirus_control_masked["category"] = dataset_freq_no_enterovirus_control_masked["category"].apply(lambda x: "Non_control" if x in ["Single AAB", "Multiple AABs", "Celiac_case"] else x)
d_f_no_enterovirus_control_masked = Dataset(
    data = dataset_freq_no_enterovirus_control_masked.drop(columns=["patient_id", "category"]),
    target = dataset_freq_no_enterovirus_control_masked["category"]
)


DATASETS = {
    "dataset_freq": d_f,
    "dataset_sum": d_s,
    "dataset_freq_no_enterovirus": d_f_no_enterovirus,
    "dataset_freq_no_enterovirus_control": d_f_no_enterovirus_control,
    "dataset_freq_no_enterovirus_control_masked": d_f_no_enterovirus_control_masked
}

# Classification

## Confusion Matrix

In [12]:
def make_confusion_matrix(y_true, y_pred, clf, title):
    cm = confusion_matrix(y_true, y_pred, )
    labels = clf.classes_
    # plot confusion matrix using plotly show numbers in each cell
    fig = px.imshow(cm, labels=dict(x="Predicted", y="True"), x=labels, y=labels, color_continuous_scale='Viridis', height=600, width=600, text_auto=True) 
    fig.update_xaxes(side="bottom")
    fig.update_layout(title=title)
    fig.show()

## MLPClassifier
Classification on all categories

In [13]:
# split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(DATASETS["dataset_freq"].data, DATASETS["dataset_freq"].target, test_size=0.2, random_state=2)

clf = MLPClassifier(hidden_layer_sizes=([300, 200, 15]), 
                    max_iter=3000,
                    activation="relu",
                    solver= "adam",
                    random_state=2,
                    verbose=False,
                    early_stopping=False,
                    learning_rate='adaptive',
                    learning_rate_init=0.002,
                    alpha=0.01,)

pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", clf)
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print("TEST Accuracy: ", accuracy_score(y_test, y_pred))
make_confusion_matrix(y_test, y_pred, clf, title = "All categories")



TEST Accuracy:  0.7808219178082192


### Control x Non Control

In [14]:
X_train, X_test, y_train, y_test = train_test_split(DATASETS["dataset_freq_no_enterovirus_control_masked"].data, DATASETS["dataset_freq_no_enterovirus_control_masked"].target, test_size=0.2, random_state=2)

classifiers = [
    LogisticRegression(random_state=42, max_iter=3000, C=0.1, solver="lbfgs", multi_class="multinomial",),
    LogisticRegression(random_state=43, max_iter=3000, C=0.1, solver="lbfgs", multi_class="multinomial"),
    #KNeighborsClassifier(n_neighbors=5),
    MLPClassifier(random_state=46, hidden_layer_sizes=([300, 200, 15]), max_iter=3000, activation="relu", solver= "adam", verbose=False, early_stopping=False, learning_rate='adaptive', learning_rate_init=0.001, alpha=0.01,),
    MLPClassifier(random_state=47, hidden_layer_sizes=([300, 200, 15]), max_iter=3000, activation="relu", solver= "adam", verbose=False, early_stopping=False, learning_rate='adaptive', learning_rate_init=0.001, alpha=0.01,),
]

clf_weights = [1, 1, 1, 1]
voting_clf = VotingClassifier(estimators=[(f"clf_{i}", clf) for i, clf in enumerate(classifiers)], voting="soft", 
                                n_jobs=-1, weights=clf_weights)

pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", voting_clf)
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print("TEST Accuracy: ", accuracy_score(y_test, y_pred))
make_confusion_matrix(y_test, y_pred, voting_clf, title = "Control vs Non_control")

TEST Accuracy:  0.863013698630137


### Non control classification

In [15]:
X_train, X_test, y_train, y_test = train_test_split(DATASETS["dataset_freq_no_enterovirus_control"].data, DATASETS["dataset_freq_no_enterovirus_control"].target, test_size=0.2, random_state=9)

classifiers = [
    #KNeighborsClassifier(n_neighbors=5, weights="distance", algorithm="auto", leaf_size=30, p=2, metric="minkowski"),
    RandomForestClassifier(n_estimators=10, criterion="gini", max_depth=None, random_state=42, bootstrap=True),
    RandomForestClassifier(n_estimators=10, criterion="entropy", max_depth=None, random_state=43, bootstrap=True),
    RandomForestClassifier(n_estimators=10, criterion="log_loss", max_depth=None, random_state=44, bootstrap=True),
    RandomForestClassifier(n_estimators=10, criterion="gini", max_depth=None, random_state=45, bootstrap=True),
    MLPClassifier(random_state=46, hidden_layer_sizes=([300, 200, 15]), max_iter=3000, activation="relu", solver= "adam", verbose=False, early_stopping=False, learning_rate='adaptive', learning_rate_init=0.001, alpha=0.01,),
    MLPClassifier(random_state=47, hidden_layer_sizes=([300, 200, 15]), max_iter=3000, activation="relu", solver= "adam", verbose=False, early_stopping=False, learning_rate='adaptive', learning_rate_init=0.001, alpha=0.01,),

]
voting_clf = VotingClassifier(estimators=[(f"clf_{i}", clf) for i, clf in enumerate(classifiers)], voting="soft", 
                            weights=[1,1,1,1, 1, 1], n_jobs=-1)

pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", voting_clf)
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print("TEST Accuracy: ", accuracy_score(y_test, y_pred))
make_confusion_matrix(y_test, y_pred, voting_clf, title = "Disease clasification")

TEST Accuracy:  0.9459459459459459


In [16]:
# display forrest feature importances

fig = go.Figure()
for i, clf in enumerate(voting_clf.estimators_[:4]):
    importances = clf.feature_importances_
    indices = np.argsort(importances)[::-1]
    fig.add_trace(go.Bar(x=DATASETS["dataset_freq_no_enterovirus_control"].data.columns[indices], y=importances[indices], name=f"{clf.__class__.__name__} {i}"))

fig.update_layout(barmode='group')
fig.update_layout(title="Feature importances for each classifier")
fig.update_yaxes(title="Importance")
fig.update_xaxes(title="Feature")
fig.update_layout(legend_title_text='Classifiers')

fig.update_xaxes(tickangle=45)
fig.show()